In [64]:
import joblib
import pandas as pd

# Carregar o modelo treinado e as colunas usadas no treino
model_xgb_tuned = joblib.load("modelo_xgb_tuned.pkl")
vectorizer_title = joblib.load("vectorizer_title.pkl")
vectorizer_desc = joblib.load("vectorizer_desc.pkl")
X_train_columns = joblib.load("X_train_columns.pkl")

print(f"✅ Modelo e colunas carregadas. Total de features esperadas: {len(X_train_columns)}")

✅ Modelo e colunas carregadas. Total de features esperadas: 110


In [65]:
# Criar DataFrame para um novo vídeo (exemplo)
novo_video = pd.DataFrame({
    "title": ["Melhor Treino para Hipertrofia"],
    "description": ["Aprenda os melhores exercícios para ganhar massa muscular rapidamente!"],
    "duration_sec": [600],  # Exemplo: 10 minutos
    "year": [2025],
    "month": [1],
    "day": [31],
    "weekday_Monday": [0],
    "weekday_Tuesday": [0],
    "weekday_Wednesday": [1],  # Se postado na quarta-feira
    "weekday_Thursday": [0],
    "weekday_Friday": [0],
    "weekday_Saturday": [0],
    "weekday_Sunday": [0]
})

# Exibir o novo vídeo
novo_video

,title,description,duration_sec,year,month,day,weekday_Monday,weekday_Tuesday,weekday_Wednesday,weekday_Thursday,weekday_Friday,weekday_Saturday,weekday_Sunday
0,Melhor Treino para Hipertrofia,Aprenda os melhores exercícios para ganhar mas...,600,2025,1,31,0,0,1,0,0,0,0


In [66]:
# 🚀 Passo 1: Criar a coluna "weekday" no mesmo formato do treino
dias_da_semana = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

# Criar a coluna "weekday" com o nome do dia correspondente
for dia in dias_da_semana:
    if f"weekday_{dia}" in novo_video.columns and novo_video[f"weekday_{dia}"].iloc[0] == 1:
        novo_video["weekday"] = dia  # Define o nome do dia corretamente

# Remover as colunas individuais de cada dia da semana, pois no treino havia apenas "weekday"
novo_video = novo_video.drop(columns=[f"weekday_{dia}" for dia in dias_da_semana if f"weekday_{dia}" in novo_video.columns])

# Exibir para verificar se a conversão deu certo
print(novo_video[["weekday"]])

     weekday
0  Wednesday


In [67]:
# 🚀 Passo 2: Adicionar as métricas faltantes para garantir compatibilidade com o treino
colunas_faltantes = ["likes", "comments", "likes_per_view", "comments_per_view", "title_word_count"]

# Preencher todas as métricas com 0 (pois o vídeo ainda não foi postado)
for col in colunas_faltantes:
    novo_video[col] = 0

# Exibir para conferir
print(novo_video[["likes", "comments", "likes_per_view", "comments_per_view", "title_word_count"]])

   likes  comments  likes_per_view  comments_per_view  title_word_count
0      0         0               0                  0                 0


In [68]:
# 🚀 Passo 3: Transformar título e descrição em TF-IDF
novo_video_title_tfidf = vectorizer_title.transform(novo_video["title"]).toarray()
novo_video_desc_tfidf = vectorizer_desc.transform(novo_video["description"]).toarray()

# Criar DataFrames com as features TF-IDF
df_title_tfidf_novo = pd.DataFrame(novo_video_title_tfidf, columns=vectorizer_title.get_feature_names_out())
df_desc_tfidf_novo = pd.DataFrame(novo_video_desc_tfidf, columns=vectorizer_desc.get_feature_names_out())

# Exibir as shapes para verificação
print(f"TF-IDF Título Shape: {df_title_tfidf_novo.shape}")
print(f"TF-IDF Descrição Shape: {df_desc_tfidf_novo.shape}")

TF-IDF Título Shape: (1, 100)
TF-IDF Descrição Shape: (1, 100)


In [69]:
# 🚀 Passo 4: Juntar os dados numéricos com os TF-IDF
novo_video_model = pd.concat([novo_video.drop(columns=["title", "description"]), df_title_tfidf_novo, df_desc_tfidf_novo], axis=1)

# 🚀 Passo 4: Remover colunas duplicadas antes de reindexar
novo_video_model = novo_video_model.loc[:, ~novo_video_model.columns.duplicated()]  # Remove colunas duplicadas

# Ajustar colunas para garantir compatibilidade com X_train
novo_video_model = novo_video_model.reindex(columns=X_train_columns, fill_value=0)

# Converter tudo para numérico
novo_video_model = novo_video_model.apply(pd.to_numeric, errors="coerce")

print(f"✅ Novo vídeo ajustado para {novo_video_model.shape[1]} features (esperado: {len(X_train_columns)})")

✅ Novo vídeo ajustado para 110 features (esperado: 110)


# AQUI COMEÇA A DAR ERRO -.-

In [70]:
# 🚀 Substituir valores NaN por 0
novo_video_model = novo_video_model.fillna(0)

# 🚀 Verificar se ainda há valores NaN
print(novo_video_model.isna().sum().sum())  # Deve mostrar 0 se tudo estiver corrigido

0


In [71]:
print(type(novo_video_model))
print(novo_video_model.shape)
print(novo_video_model.head())

<class 'pandas.core.frame.DataFrame'>
(1, 110)
   likes  comments  duration_sec  year  month  day  weekday  likes_per_view  \
0      0         0           600  2025      1   31      0.0               0   

   comments_per_view  title_word_count  ...  todo  toguro  top  tortuguita  \
0                  0                 0  ...   0.0     0.0  0.0         0.0   

   trabalho  treinar   um  usar  você   vs  
0       0.0      0.0  0.0   0.0   0.0  0.0  

[1 rows x 110 columns]


In [72]:
# 🚀 Converter o DataFrame para NumPy antes da previsão
previsao_visualizacoes = model_xgb_tuned.predict(novo_video_model.to_numpy())

# 🚀 Exibir a previsão
print(f"📊 Previsão de Visualizações: {int(previsao_visualizacoes[0])}")

📊 Previsão de Visualizações: 17367
